# Saving Multiple Instruments to a Data File

In [2]:
import requests
import pandas as pd
import json
from dateutil import parser

Repeating some steps from notebook 01-Fetching-Data-from-Oanda-API-Endpoints

In [3]:
# get the account access data
account_data = pd.read_csv("~/Documents/temp/oanda/testnet_key.txt", sep=" ", header=None)
API_KEY = account_data[0][2]
ACCOUNT_ID = account_data[0][1]
OANDA_URL = account_data[0][0]

In [5]:
# start session
session = requests.Session()

In [24]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [25]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [26]:
# Fetching Data from the Instruments Endpoint
url_inst = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"
url_inst

'https://api-fxpractice.oanda.com/v3/accounts/101-012-22793269-001/instruments'

In [27]:
response = session.get(url_inst, params=params, data=None, headers=None)
response.status_code

200

In [30]:
data = response.json() # dict_keys(['instruments', 'lastTransactionID'])
instruments_list = data["instruments"]

In [32]:
len(instruments_list)

129

In [33]:
# list of keys we're intersted in getting 
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 
         'tradeUnitsPrecision', 'marginRate']

In [38]:
instruments_dict = {}
for i in instruments_list:
    # print(i['name'])
    key = i['name']
    instruments_dict[key] = {
        k: i[k] for k in key_i
    }

In [45]:
# Fetching a candle pair
def fetch_candles(api_url, pair_name, count="10", granularity="H1"):
    url = f"{api_url}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    status = response.status_code
    
    if status == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    
    return status, data

In [46]:
# Saving the fetched Candles in a Data File
def create_data_file(api_url, pair_name, count="10", granularity="H1"):
    
    status, data = fetch_candles(
        api_url, 
        pair_name, 
        count=count, 
        granularity=granularity
    )
    
    if status != 200:
        print("Failed", pair_name, data)
        return    
    
    if len(data) == 0:
        print("No candles", pair_name)
        return
    
    candles_df = get_candles_df(data)
    # safe compressed data frame to file
    candles_df.to_pickle(f"Data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [49]:
# collecting certain info from the instruments
# and converting it to a data frame
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    final_data = []
    
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        # loop through the prices and make for each one a column key 
        # that looks like mid_c, mid_o, mid_h, mid_c
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

In [50]:
# list of tradeable currencies
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [52]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(OANDA_URL, pr, count=4001, granularity=g)

EUR_USD H1 4000 candles, 2021-11-30 19:00:00+00:00 2022-07-21 12:00:00+00:00
EUR_USD H4 4000 candles, 2019-12-26 22:00:00+00:00 2022-07-21 09:00:00+00:00
EUR_GBP H1 4000 candles, 2021-11-30 17:00:00+00:00 2022-07-21 12:00:00+00:00
EUR_GBP H4 4000 candles, 2019-12-26 22:00:00+00:00 2022-07-21 09:00:00+00:00
EUR_JPY H1 4000 candles, 2021-11-30 11:00:00+00:00 2022-07-21 12:00:00+00:00
EUR_JPY H4 4000 candles, 2019-12-26 14:00:00+00:00 2022-07-21 09:00:00+00:00
EUR_CHF H1 4000 candles, 2021-11-30 20:00:00+00:00 2022-07-21 12:00:00+00:00
EUR_CHF H4 4000 candles, 2019-12-26 22:00:00+00:00 2022-07-21 09:00:00+00:00
EUR_NZD H1 4000 candles, 2021-11-30 19:00:00+00:00 2022-07-21 12:00:00+00:00
EUR_NZD H4 4000 candles, 2019-12-26 22:00:00+00:00 2022-07-21 09:00:00+00:00
EUR_CAD H1 4000 candles, 2021-11-30 20:00:00+00:00 2022-07-21 12:00:00+00:00
EUR_CAD H4 4000 candles, 2019-12-26 22:00:00+00:00 2022-07-21 09:00:00+00:00
EUR_AUD H1 4000 candles, 2021-11-30 18:00:00+00:00 2022-07-21 12:00:00+00:00